In [1]:
import numpy as np
import pandas as pd 
import sys
sys.path.append('..')
sys.path.append('../..')
from pandas import *
from pandas import DataFrame
from pandas import Series
import scipy
from scipy import optimize
from scipy.optimize import minimize, rosen, rosen_der
import math
from pandas.tslib import Timestamp
from datetime import datetime
import GC_risk_parity
from GC_risk_parity import *
from numpy.random import randn

/Users/wrk/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  del sys.path[0]


In [2]:
CP = pd.DataFrame(pd.read_csv('data_original.csv'))
CP.index = CP['date']
CP = CP.drop(['date'],axis = 1)
cur_len = len(CP)
data = []

## set your backtest frequency: f_month for monthly and f_quarter for quarterly
freq = f_month 

times =  int(cur_len/f_month)

for x in range(times):
    temp = CP.iloc[freq*x:freq*(x + 1)]
    data.append(np.matrix(temp).T)


record = [] ## record results of each optimize

IOError: File data_original.csv does not exist

In [3]:
for x in range(times):
    Cov_matrix = np.cov(data[x])
    sigma = []
    for i in range(Num_of_assets):
        sigma.append ( Cov_matrix[i][i] )
        
        
    def fun(w):
 ## w = [w0,w1,w2,w3,w4,w5,w6] w_matrix: turn w into matrix
        w_matrix = np.matrix(w)
        p = [None] * Num_of_assets
        Sigma =  w_matrix * Cov_matrix * w_matrix.T
        for i in range(Num_of_assets):
            add = 0   
            for j in range(Num_of_assets):        
                add += w[j]*w[i]*sigma[j]*sigma[i]    
            p[i] = add/(Sigma)
    ## p[i] = risk contribution ratio for each stock_i

    ## our goal: minimize f:
        
        minimize_solve = ((p[0] + p[1] +p[2] -(p[3]+[4]))**2 + (p[3] + p[4] - (p[5]+p[6]))**2 + (p[0]+p[1]+p[2]-(p[5]+p[5]))**2)
        return minimize_solve
    
    
    def constrain_sum_of_ratio(w):
    
        total_sum = 0.0

        for i in range(Num_of_assets):
            total_sum += float(w[i])

            total_sum = float(total_sum - 1.0)

        return total_sum

    weight_init = [w0, w1, w2, w3, w4, w5, w6]
    bnds = (b0,b1,b2,b3,b4,b5,b6)
    cons = ({'type': 'eq', 'fun': constrain_sum_of_ratio})
    
    res = minimize(fun, weight_init, method='L-BFGS-B', bounds=bnds,constraints=cons)
    record.append(res.x)

NameError: name 'times' is not defined

In [154]:
print record

[array([  7.29570175e-04,   0.00000000e+00,   3.45413754e-03,
         1.00000000e+00,   1.00000000e+00,   1.78637030e-03,
         9.99396757e-01]), array([ 0.00112641,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ]), array([  9.56331655e-04,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00]), array([  6.77222824e-03,   4.73456202e-05,   4.73642121e-05,
         1.00000000e+00,   1.00000000e+00,   1.57857119e-03,
         3.07066747e-01]), array([  9.38310589e-04,   2.13627580e-04,   2.88412698e-04,
         1.00000000e+00,   9.99988548e-01,   2.50571211e-03,
         3.81422043e-02]), array([  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -6.93889390e-18,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), array([  5.44254449e-03,   3.39290239e-05,   0.00000000e+00,
         9.99996878e-01,   1.00000000e+00,   3.15406622e-03,
         0.00000000e+00])